In [8]:
import numpy as np
X = np.random.random((1000, 3))

In [9]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))
%timeit pairwise_numpy(X)

10 loops, best of 3: 32 ms per loop


In [10]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

%timeit pairwise_python(X)

1 loops, best of 3: 3.38 s per loop


In [11]:
from numba import double
from numba.decorators import jit, autojit

pairwise_numba = autojit(pairwise_python)

%timeit pairwise_numba(X)

The slowest run took 40.07 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 5.03 ms per loop


In [14]:
%load_ext cythonmagic
import cython

The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


In [15]:
%%cython
import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

ERROR: Cell magic `%%cython` not found.


In [18]:
import pandas as pd

df = pd.DataFrame({'a': np.random.randn(1000),
                   'b': np.random.randn(1000),
                   'N': np.random.randint(100, 1000, (1000)),
                   'x': 'x'})

In [19]:
def f(x):
    return x * (x - 1)


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [20]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

10 loops, best of 3: 162 ms per loop


In [21]:
%prun -l 4 df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

In [22]:
import numba

@numba.jit
def f_plain(x):
   return x * (x - 1)

@numba.jit
def integrate_f_numba(a, b, N):
   s = 0
   dx = (b - a) / N
   for i in range(N):
       s += f_plain(a + i * dx)
   return s * dx

@numba.jit
def apply_integrate_f_numba(col_a, col_b, col_N):
   n = len(col_N)
   result = np.empty(n, dtype='float64')
   assert len(col_a) == len(col_b) == n
   for i in range(n):
      result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
   return result

def compute_numba(df):
   result = apply_integrate_f_numba(df['a'].values, df['b'].values, df['N'].values)
   return pd.Series(result, index=df.index, name='result')

In [26]:
%timeit compute_numba(df)
%prun -l 4 compute_numba(df)

1000 loops, best of 3: 614 µs per loop
 

In [24]:
import numba

def double_every_value_nonumba(x):
    return x*2

@numba.vectorize
def double_every_value_withnumba(x):
    return x*2

In [25]:
%timeit df['col1_doubled'] = df.a.apply(double_every_value_nonumba)
%timeit df['col1_doubled'] = df.a*2
%timeit df['col1_doubled'] = double_every_value_withnumba(df.a.values)

1000 loops, best of 3: 635 µs per loop
10000 loops, best of 3: 178 µs per loop
The slowest run took 165.82 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 127 µs per loop
